In [1]:
# Скачивание датасета
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz --output data/flowers.tgz
!tar -xvzf data/flowers.tgz -C data/ -h

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  25.0M      0  0:00:08  0:00:08 --:--:-- 28.3M
tar: Option -L is not permitted in mode -x


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = 'data/flower_photos'
model_path = 'best_vgg16_flowers.pth'
num_epochs = 10
num_classes = 5
learning_rate=0.001
best_accuracy = 0.0

### Train model

In [3]:
# Преобразования данных (например, изменение размера и нормализация)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_split = 0.8
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
train_size = int(train_split * len(image_datasets))
val_size = len(image_datasets) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, val_size])

# Создаем DataLoader'ы для обучения и валидации
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
num_classes = len(image_datasets.classes)

# Загрузка предобученной модели VGG-16
model = models.vgg16(pretrained=True)

# Заморозим все слои, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Изменим последний классификатор под количество классов в нашем датасете
model.classifier[6] = nn.Linear(4096, num_classes)
model = model.to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=learning_rate)

# Функция для обучения модели
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
    
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Включаем режим обучения
            else:
                model.eval()   # Включаем режим оценки
    
            running_loss = 0.0
            running_corrects = 0
    
            # Проходим по батчам данных
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
    
                # Обнуляем градиенты
                optimizer.zero_grad()
    
                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
    
                    # Обратное распространение и оптимизация только на обучении
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
    
                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
    
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Обучаем модель
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

# Сохраняем модель в файл
torch.save(model.state_dict(), model_path)



/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.6318 Acc: 0.7606
val Loss: 0.4382 Acc: 0.8406
Epoch 1/9
----------
train Loss: 0.4611 Acc: 0.8297
val Loss: 0.4226 Acc: 0.8515
Epoch 2/9
----------
train Loss: 0.4093 Acc: 0.8495
val Loss: 0.4342 Acc: 0.8392
Epoch 3/9
----------
train Loss: 0.4084 Acc: 0.8501
val Loss: 0.4269 Acc: 0.8542
Epoch 4/9
----------
train Loss: 0.3859 Acc: 0.8597
val Loss: 0.4100 Acc: 0.8624
Epoch 5/9
----------
train Loss: 0.3709 Acc: 0.8699
val Loss: 0.4009 Acc: 0.8638
Epoch 6/9
----------
train Loss: 0.3569 Acc: 0.8726
val Loss: 0.3984 Acc: 0.8638
Epoch 7/9
----------
train Loss: 0.3687 Acc: 0.8600
val Loss: 0.3931 Acc: 0.8733
Epoch 8/9
----------
train Loss: 0.3569 Acc: 0.8678
val Loss: 0.3918 Acc: 0.8624
Epoch 9/9
----------
train Loss: 0.3850 Acc: 0.8566
val Loss: 0.3962 Acc: 0.8747


### Inference model

In [4]:
model = models.vgg16(pretrained=False)  # Создаем модель VGG-16 без предобученных весов
model.classifier[6] = nn.Linear(4096, num_classes)  # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path)) # Загружаем обученные веса
model = model.to(device)
model.eval()

/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/3127508217.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allo

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
%%time

# Функция для выполнения инференса
def inference(model, inputs):
    inputs = inputs.to(device)
    with torch.no_grad():  # Отключаем градиенты для инференса
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    return preds.cpu().numpy()  # Возвращаем предсказания в формате numpy

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    # Получаем предсказания модели
    preds = inference(model, inputs)

    # Сравнение предсказанных классов с истинными
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8706
CPU times: user 11min 13s, sys: 1min 38s, total: 12min 52s
Wall time: 3min 13s


### Save model to ONXX

In [6]:
model = models.vgg16(pretrained=False)  # Создаем модель VGG-16 без предобученных весов
model.classifier[6] = torch.nn.Linear(4096, num_classes)  # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path)) # Загружаем обученные веса
model = model.to(device)
model.eval()  # Устанавливаем режим оценки

# Создаем фиктивный входной тензор (batch size 1, 3 канала, 224x224 пикселей)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Путь для сохранения ONNX модели
onnx_model_path = 'vgg16_flowers.onnx'

# Экспортируем модель в формат ONNX
torch.onnx.export(
    model,                     # Модель для экспорта
    dummy_input,               # Пример входного тензора
    onnx_model_path,           # Имя выходного файла
    export_params=True,        # Экспортируем обученные параметры
    opset_version=11,          # Версия ONNX opset
    do_constant_folding=True,  # Оптимизация неизменяемых частей
    input_names=['input'],     # Имя входного тензора
    output_names=['output'],   # Имя выходного тензора
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})  # Динамическое изменение размера батча

print(f"Модель VGG-16 успешно экспортирована в {onnx_model_path}")


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/2472523256.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))

Модель VGG-16 успешно экспортирована в vgg16_flowers.onnx


### Inference model in ONXX

In [7]:
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_model_path)

In [8]:
%%time

# Функция для инференса
def onnx_inference(ort_session, inputs):
    ort_inputs = {ort_session.get_inputs()[0].name: inputs}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = onnx_inference(ort_session, inputs_numpy)
    preds = np.argmax(outputs[0], axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8610
CPU times: user 14min 34s, sys: 1min, total: 15min 35s
Wall time: 4min 25s


### Save model in tensorflow ONXX

In [9]:
from onnx_tf.backend import prepare
import onnx

# Загружаем модель в формате ONNX
onnx_model = onnx.load(onnx_model_path)

# Конвертируем ONNX модель в TensorFlow формат
tf_rep = prepare(onnx_model)
tf_model_path = 'vgg16_flowers_tf_model'
tf_rep.export_graph(tf_model_path)
print(f"Модель экспортирована в TensorFlow формат: {tf_model_path}")


/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_53_x, mul_3_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: vgg16_flowers_tf_model/assets


INFO:tensorflow:Assets written to: vgg16_flowers_tf_model/assets
INFO:absl:Writing fingerprint to vgg16_flowers_tf_model/fingerprint.pb


Модель экспортирована в TensorFlow формат: vgg16_flowers_tf_model


### Inference model in tensorflow ONXX

In [10]:
import tensorflow as tf

# Загрузка модели TensorFlow
model = tf.saved_model.load(tf_model_path)

In [11]:
%%time

# Выполняем инференс
def tensorflow_inference(model, inputs):
    inputs_tf = tf.convert_to_tensor(inputs, dtype=tf.float32)
    outputs = model.signatures['serving_default'](inputs_tf)['output']
    return outputs

correct = 0
total = 0
for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = tensorflow_inference(model, inputs_numpy)
    preds = np.argmax(outputs, axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8569
CPU times: user 17min 53s, sys: 1min 49s, total: 19min 42s
Wall time: 3min 55s


_Inference model:_ <br/>
Точность (Accuracy): 0.8706 <br/>
Время обработки: 12 минут 52 секунды (время CPU) и 3 минуты 13 секунд (время "на стене"). <br/>
Модель показывает наилучшую точность по сравнению с другими вариантами и является наиболее эффективной по времени обработки на стене.

_Inference model в ONNX:_<br/>
Точность: 0.8610 <br/>
Время обработки: 15 минут 35 секунд (время CPU) и 4 минуты 25 секунд (время "на стене"). <br/>
ONNX-версия модели несколько снижает точность (на 0.0096), однако затрачивает больше времени на обработку, что может быть не столь эффективным.

_Inference model в TensorFlow ONNX:_<br/>
Точность: 0.8569 <br/>
Время обработки: 19 минут 42 секунды (время CPU) и 3 минуты 55 секунд (время "на стене"). <br/>
Эта версия показывает самую низкую точность (на 0.0137 меньше, чем оригинальная модель), при этом время CPU оказывается самым долгим. Время на стене чуть лучше, чем у ONNX-версии, но по всем показателям эта модель уступает.

Оригинальная модель демонстрирует лучшую точность (0.8706) и быстрое время выполнения. <br/>
Вариант на TensorFlow ONNX демонстрирует самую низкую точность и длительное время обработки. <br/>
__Рекомендуется использовать оригинальную версию модели для повышения точности и скорости__

